In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from util.util import *
from util.drawer import *
from strategy.Bolling_minute import *
from strategy.Runner import Runner
from strategy.Strategy import Strategy

from pyecharts import options as opts

In [ ]:
day_path = '/Users/jinkun.tian/gitee/financial_lab/strategy/data/price/day/'
min_path = '/Users/jinkun.tian/gitee/financial_lab/strategy/data/price/minute/'
file_name = '601088.csv'

df_d = pd.read_csv(day_path + file_name)
df_d.rename(columns={'Unnamed: 0':'date'},inplace=True)
df_m = pd.read_csv(min_path + file_name)
df_m.rename(columns={'Unnamed: 0':'time'},inplace=True)

In [ ]:
# 初始化策略
Strategy.init()
bollingV3 = BollingV3()

# 调用running
runner = Runner(1)
runner.df = df_d
runner.df_m = df_m

runner.strategys.append(bollingV3)

runner.running_m()

trades = Strategy.global_context['trades']

In [ ]:
trades

In [ ]:
k_graph = draw_K(df_d,trades=trades)
k_graph.render_notebook()

In [ ]:
def get_trades_df(df,trades):
    trades_df = pd.DataFrame(trades)
    trade_data = pd.merge(df['date'],trades_df[['sell_day','profit','own_money']],how='left',left_on='date',right_on='sell_day',left_index=False, right_index=False, )
    
    last_profit = None
    last_own_money = Strategy.global_context['start_money']
    for i in trade_data.index:
        if pd.isna(trade_data.loc[i,'profit']):
            trade_data.loc[i,'profit'] = last_profit
        else:
            last_profit = trade_data.loc[i,'profit']

        if pd.isna(trade_data.loc[i,'own_money']):
            trade_data.loc[i,'own_money'] = last_own_money
        else:
            last_own_money = trade_data.loc[i,'own_money']
    return trade_data

In [ ]:
trade_1_df = get_trades_df(df_d,trades)

In [ ]:
line_1= get_line(trade_1_df,'own_money','算方法1')
line_1.render_notebook()

In [ ]:
trade_0_df = get_trades_df(df_d,trades_0)
line_0= get_line(trade_0_df,'own_money','算方法0')
line_1.overlap(line_0)
line_1.render_notebook()


In [ ]:
line_1.set_global_opts(datazoom_opts=[  # 缩放选项
        opts.DataZoomOpts(
            is_show=False,
            type_="inside",
            xaxis_index=[0, 1],
            # 初始的框选范围
            range_start=80,
            range_end=100,
        ),
        opts.DataZoomOpts(
            is_show=True,
            xaxis_index=[0, 1],
            type_="slider",
            pos_top="95%",
            range_start=80,
            range_end=100,
        )
    ])
line_1.render_notebook()


In [ ]:
trades_df = pd.DataFrame(trades)
trade_data = pd.merge(df_d['date'],trades_df[['sell_day','profit','own_money']],how='left',left_on='date',right_on='sell_day',left_index=False, right_index=False, )
